# Lecture 1. Language Models, the Chat Format and Tokens

## 1. Setup

#### 1.1. Install required modules

In [ ]:
#!pip install -U python-dotenv

#!pip install -U openai

#!pip install -U tiktoken

#### 1.2. Load the API key and relevant Python libaries.

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#### 1.3. A helper function to conveniently access OpenAI's LLM

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

## 2. Introduction to Large Language Models

#### 2.1 How Large Language Models work

You're probably familiar with the **text generation process** where you can **give a prompt**,  
"I love eating", and **ask an LLM to fill** in what the things are **likely completions**  
given this prompt. And it may say, "Bagels with cream cheese, or my mother's meatloaf, or out  
with friends".

![Large Language Models](./Images/llm_intro.png)

But how did the model learn to do this? The main tool used to train an LLM  
is actually **supervised learning**.

#### 2.2 Supervised Learning

 In supervised learning, a computer learns an **input-output** or **X or Y mapping**  
 using **labeled training data**. So for example, if you're using supervised learning  
 to learn to classify the sentiment of restaurant reviews, you might collect a training  
 set like this, where a review like, "The pastrami sandwich is great!", is labeled as a  
 positive  sentiment review, and so on. And "Service was slow, the food was so-so.",  
 it was negative, and "The earl grey tea was fantastic.", has a positive label. 

The process for **supervised learning** is typically
 
 - get labeled data
 -  train AI model on data
 - deploy the model and call it
 
If we give it a new restaurant review like **best pizza I've ever had**. You  
hopefully output that as a **positive sentiment**.

![Supervised Learning](./Images/supervised_learning.png)

#### 2.3 LLM using Supervised Learning

It turns out that **supervised learning is a core building block** for **training Large Language Models**.  
Specifically, a **Large Language Model can be built by using supervised learning to repeatedly predict the**  
**next word**. Let's say that in your training sets of a lot of text data, you have to sentence, "My favorite food  
is a bagel with cream cheese and lox.". Then this sentence is turned into a sequence of training examples,  
where given a sentence fragment, "My favorite food is a", if you want to predict the next word in this case was  
"bagel", or given the sentence fragment or sentence prefix, "My favorite food is a bagel", the next word in this  
case would be "with", and so on. And given a large training set of hundreds of billions or sometimes even more  
words, you can then create a massive training set where you can start off with part of a sentence or part of a  
piece of text and repeatedly ask the language model to learn to predict what is the next word. 

![LLM form Supervised Learning](./Images/llm_from_supervised_learning.png)

#### 2.4 Types of LLM

There are broadly two major types of Large Language Models.

 - Base LLM
 
   The base LLM repeatedly predicts the next word based on text training data. And so if I give it a  
   prompt, "Once upon a time there was a unicorn", then it may, by repeatedly predicting one word at a  
   time, come up with a completion that tells a story about a unicorn living in a magical forest with  
   all her unicorn friends. A downside of this is that if you were to prompt it with "What is the capital  
   of France?", quite possible that on the internet there might be a list of quiz questions about France.  
   So it may complete this with "What is France's largest city, what is France's population?", and so on. 
   
 - Instruction Tuned LLM

   Instruction Tuned LLM instead tries to follow instructions and will hopefully say, "The capital of France  
   is Paris."

![Types of LLMs](./Images/types_of_llms.png)

#### 2.5 How to get Instruction Tuned LLM from Base LLm



![Instruction Tuned LLM from Base LLm](./Images/base_llm_to_instruction_tuned_llsm.png)

## 3. Some example sentence completition using LLm

In [ ]:
response = get_completion("What is the capital of France?")

In [ ]:
# print(response)
# The capital of France is Paris.

In [ ]:
response = get_completion("Take the letters in lollipop \
and reverse them")

In [ ]:
# print(response)
# pillipol

It looks like the LLM failed to follow the instruction in the second case, which appears  
to be a simple task. It turns out that there's one more important detail for how a Large  
Language Model works, which is it doesn't actually repeatedly predict the next word, it  
instead repeatedly predicts the **next token**. 

#### 3.1. Tokens

An LLM takes a sequence of characters, like "Learning new things is fun!", and group the  
characters together to form tokens that comprise **commonly occurring sequences of characters**.  
if you were to give it the word **lollipop**, the tokenizer actually breaks this down into three  
tokens, **"l"**, **"oll"** and **"ipop"**. And because LLM isn't seeing the individual letters,  
is instead seeing these three tokens, it's more difficult for it to correctly print out these  
letters in reverse order.

If you pass it lollipop with dashes in between the letters, like **L-O-L-L-I-P-O-P** it tokenizes  
each of these characters into an individual token, making it easier for it to see the individual  
letters and print them out in reverse order.

For an LLM

 - Input is often called **context**

 - Output is often called **target** or **completion**

 - The goal is to predict the target given the context

![Tokens](./Images/tokens.png)

In [ ]:
response = get_completion("Take the letters in l-o-l-l-i-p-o-p \
and reverse them")

In [ ]:
# print(response)
# p-o-p-i-l-l-o-l

## 4. System, User and Assistant Messages

The **system message** specifies the **overall tone** of what you want the **Large Language Model**  
or **the assistant**to do, and the **user message** is a **specific instruction** that you wanted to carry  
out given this higher level behavior that was specified in the system message. 

![System, User and Assistant Messages](./Images/system_user_and_assistant.png)

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

#### 4.1. An example completion

In [ ]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)

In [ ]:
# print(response)

# Oh, the happy carrot, so bright and cheery,
# With a vibrant orange hue, never weary.
# In the garden it grew, bathed in sun's gleam,
# Bringing joy and laughter, like in a dream.

# Its leafy green tops danced in the breeze,
# While the carrot below stood tall with ease.
# It had a big smile, so silly and grand,
# Bringing happiness to all the land.

# From the earth it was pulled, with a merry sound,
# Into a salad, its purpose was found.
# Oh, the happy carrot, so full of delight,
# Spreading joy with every bite.

# So remember, my friend, in your daily fare,
# A happy carrot's love is always there.
# Bring a smile to your face, let laughter ensue,
# With a happy carrot, your day will be true.

#### 4.2. An example completion with length restriction

In [ ]:
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)

In [ ]:
# print(response)

# Once upon a time, there was a cheerful carrot named Charlie who grew up alongside a field of
# vibrant sunflowers, spreading joy wherever he went.

#### 4.3. An example completion with a given tone and length restriction

In [ ]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)

In [ ]:
# print(response)
# Once there was a happy carrot named Larry, who lived in a garden full of joy and was always merry.

#### 4.3. A helper function which returns token count along with completion

In [ ]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
        'prompt_tokens':response['usage']['prompt_tokens'],
        'completion_tokens':response['usage']['completion_tokens'],
         'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [ ]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [ ]:
# print(response)

# Oh, the happy carrot, so bright and orange,
# Grown in the garden, a joyful forage.
# With a smile so wide, from top to bottom,
# It brings happiness, oh how it blossoms!

# In the soil it grew, with love and care,
# Nourished by sunshine, fresh air to share.
# Its leaves so green, reaching up so high,
# A happy carrot, oh my, oh my!

# With a crunch and a munch, it's oh so tasty,
# Filled with vitamins, oh so hasty.
# A happy carrot, a delight to eat,
# Bringing joy and health, oh what a treat!

# So let's celebrate this veggie so grand,
# With a happy carrot in each hand.
# For in its presence, we surely find,
# A taste of happiness, one of a kind!

# print(token_dict)
# {'prompt_tokens': 37, 'completion_tokens': 164, 'total_tokens': 201}